# Decision Trees

[Geron notebook](external/handson-ml/06_decision_trees.ipynb)

[VanderPlass notebook](external/PythonDataScienceHandbook/notebooks/05.08-Random-Forests.ipynb)

[scipy 2018](external/scipy-2018-sklearn/notebooks/18.In_Depth-Trees_and_Forests.ipynb)

- Revisit Titanic Using decision trees
    - code already done in [Non_Numerical_Data notebook](http://localhost:8888/notebooks/NYU/Non_Numerical_Data.ipynb#Random-Forest-Classifier)
    
[Geron equations](external/handson-ml/book_equations.ipynb#Chapter-6)

[Geron ensemble and RandomForest](external/handson-ml/07_ensemble_learning_and_random_forests.ipynb)

**NEED PICTURE**
- Colored + and - in 2D
- Show first split on one variable, and subsequent splits
    - complicted decision boundary

Need to compute Purity of each variable that can be used to split
- use pandas to create the 2 groups created by the split
- count number in each class

Plan
- logic function to generate data
- algo in English with abstract "Choose variable" step
- describe recursive
- use pandas slice/dice to isolate nodes
    - get counts
- use graphviz to generate slides
- use Gini w/o justification
    - Geron page 218 Gini
    - Warning page 219: sklearn used CART, which is a binary tree (node has 2 children)
        - other algos use ID3 which can have multiple children per node
    - page 219: DecisionTrees highly interepretable; Forests not
    - page 221: notation chose feature $k$ and threshold $t_k$ to do split
        - greedy algo, so may not be best
- optional features
    - additional	stopping	conditions	( min_samples_split ,
min_samples_leaf ,	 min_weight_fraction_leaf ,	and	 max_leaf_nodes
- come back and use Info Gain
    - entropy
    - `DecisionTreeClassifier` choices: gini, entropy
    - `DecisionTreeRegressor` choices: mse, mae
        - variance reduction of split
- Problems with trees
- Bagging to overcome
    - out of bag
- Random forest
    - bootstrap multiple datasets
        - choose subset of variables
            - weak learner
- Boosting
    - combine weak learners
        - any model
        - use stumps as example

Decision tree examples
[compare classifiers](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py)
- use Decision Tree on Moons and Linear Separable data
- Titanic

geron


average number of bits in one sample  
cross entropy   depends on message  encoding  
average message  length
goal  is to make messages  length  closer  to entropy  
@5:00 variable  length  message  
Huffman  coding  ?
message  length is optimal for some other  distribution  
so cross entopy gain measures  distance  between 2 distribution ? 
@7:00 cross entropy  like entropy  
true prob times other distribution  
so difference cross  entropy  is how m6ch closer we move target distribution  to true distribution 
KL divergence  
cross entropy  equals entropy  when identical  distribution  
one hot as true target distribution  
so minimize  KL for objective  of classification  
entropy  of one hot  is 0
size of message  vs amount  of info
negative  log is log of reciprocal  
difference  log  in  log ratio  
so gain is log ratio
multinomial logitistc  classification  
one hot target  vs one against all
match distribution 
log softmax function  is cr8ss entropy ! 
since softmax is the probability  
cross  entropy  better yhan accuracy  because  it weights with true probability  
accuracy  is unweighted  
information  gain
Google  ml develop decision  trees 
@5:00
gain  is reduction  in gini
relation to shannon bits?


# Entropy

We will study Entropy for two reasons
- an alternative to Gini for deciding which variable/what threshold to use in constructing a Decision Tree
- a Cost measure for Classification (Cool Cost Function ?)

- A measure of the randomness of a distribution
- Interpreted as the number of bits of information obtained from a single sample

## What is a bit ?
- Amount of information need to reduce uncertainly by a factor of 2
- Examples

### Example: what is a bit

Here, a bit is the amount of information **not** a length of the message

### Average number of bits in a sample

## Information gain
- Reduction in entropy after having received some bits, compared to before receiving the bits

## Cross entropy
Measure of distance between two distributions.

In Particular, One-Hot encoded target is a distribution with a single 1.

Can use Cross Entropy as cost function in Multiomial Classification (don't have to use Logistic to classify)

# YouTube Statquest: Josh Stamner 
- Entire sequence of videos excellent
    - Decision Tree
    - Random Forest parts 1, 2
    - boosting (on trees, stumps)
    
- random forest
    - bagging
        - like cross validation
        - bootstrap sample of observations (the "bag") from training set; unused observatios are called Out of Bag
        - fit model on bag; evaluate on Out of Bag
        - single prediction is average of predictions across the model fit on each bag
            - and accuracy of an observation is evaluated
                - only on models in which observation is out of bag

## YouTube  Google  developers ML recipes  
[link](https://www.youtube.com/watch?v=cKxRvEZd3Mw&list=PLOU2XLYxmsIIuiBfYad6rFYQU_jL2ryal)

[visualize data with Facets](https://pair-code.github.io/facets/?utm_campaign=ai_series_githubfacets_103017&utm_source=gdev&utm_medium=yt-desc%5C)

decision  tree
visualize   sklearn tools
algo from scratch 
reduce  uncertainty  
what makes a good  feature  
stacked histogram 

recipe  4 tensorflow playground 

intro to feature  engineering  in tf
age vs income  
bucket as category  
cross product features  , interactions 
easy for categorical 

weka
accuracy  limitation  
confusion  matrix  
rank feature  by info gain like decision  tree

In [1]:
# My standard magic !  You will see this in almost all my notebooks.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

%matplotlib notebook

In [2]:
# %load code/titanic.py
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Common imports
import os

# Sklearn imports

TITANIC_PATH = os.path.join("./external/jack-dies", "data")

train_data = pd.read_csv( os.path.join(TITANIC_PATH, "train.csv") )
test_data  = pd.read_csv( os.path.join(TITANIC_PATH, "test.csv")  )

# Numeric pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

from sklearn.pipeline import Pipeline
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer

num_features = ["Age", "SibSp", "Parch", "Fare"]
num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(num_features)),
        ("imputer", SimpleImputer(strategy="median")),
    ])

# Categorical pipeline
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

class SexToInt(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        sex = X["Sex"]
        X["Sex"] = 0
        X[ sex == "female" ] = 1
        
        return(X)

cat_features =  ["Pclass", "Sex"]

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(cat_features)),
        ("imputer", MostFrequentImputer()),
        ("sex_encoder", SexToInt() ),
    ])

# Union the numberic and categorical pipelines
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

# Run the pipelinem return an ndarray
X_train = preprocess_pipeline.fit_transform(train_data)


# Extract the training target
target = "Survived"
y_train = train_data[target]

# Create models
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")

from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

from sklearn import linear_model, preprocessing, model_selection 

logistic_clf = linear_model.LogisticRegression()

from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)


num_folds = 10

# Train models
for name, clf in { "Logistic": logistic_clf,
                   "SVM": svm_clf,
                   "Decision Tree": tree_clf,
                   "Random Forest": forest_clf
                 }.items():
    
    print("Model: ", name)
    clf.fit(X_train, y_train)
    
    X_test = preprocess_pipeline.transform(test_data)
    y_pred = clf.predict(X_test)

    scores = cross_val_score(clf, X_train, y_train, cv=num_folds)
    print("\tMean score ({nf}-fold validation: {s:0.3f}".format(
            nf=num_folds,
            s=scores.mean()
            )
          )


Model:  Logistic


/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

	Mean score (10-fold validation: 0.789
Model:  SVM


/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kjp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Spec

...

	Mean score (10-fold validation: 0.732
Model:  Decision Tree


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

	Mean score (10-fold validation: 0.780
Model:  Random Forest


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

	Mean score (10-fold validation: 0.802


In [3]:
proj = "titanic"

feature_names = num_features.copy()
feature_names.extend(cat_features)

target_names = [ "No", "Yes"]
feature_names
target_names

['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'Sex']

['No', 'Yes']

In [4]:


from sklearn.tree import export_graphviz

PROJECT_ROOT_DIR="/tmp"

out_file=os.path.join(PROJECT_ROOT_DIR,"{proj}_tree".format(proj=proj)) 

export_graphviz(
        tree_clf,
        out_file=out_file + ".dot",
        feature_names=feature_names,
        class_names=target_names,
        rounded=True,
        filled=True
    )

## n.b., can't seem to access files outside of Jupyter root, so have to create image in ./images

In [5]:
cmd = "dot -Tpng {dotf} -o {pngf}".format(dotf=out_file + ".dot", pngf="images/" + proj + "_tree.png")
cmd

import subprocess
retval = subprocess.call(cmd, shell=True)
retval

'dot -Tpng /tmp/titanic_tree.dot -o images/titanic_tree.png'

0

![]("/tmp/titanic_tree.png")

![titantic](images/titanic_tree.png)


In [9]:
X_train.shape


(891, 6)

In [45]:
import decision_trees_helper as dthelp
%aimport decision_trees_helper

th_logit = dthelp.TitanicHelper()

target_name = "Survived"

In [46]:
logit_clf = th_logit.make_logit_clf()

In [47]:
th_logit.fit(logit_clf, train_data, target_name)
th_logit.scores

SexToInt:transform: Cheating alert!, X has 2 columns.


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

array([0.77653631, 0.81005587, 0.7752809 , 0.75280899, 0.81355932])

In [66]:
th_dt = dthelp.TitanicHelper()

tree_clf = th.make_tree_clf()
th_dt.fit(tree_clf, train_data, target_name)
th_dt.scores

SexToInt:transform: Cheating alert!, X has 2 columns.


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

array([0.80446927, 0.80446927, 0.78651685, 0.75280899, 0.78531073])

In [68]:
th_dt.export_tree(tree_clf, "dt", th_dt.feature_names, [ "No", "Yes"] )

{'dot_file': 'dt.dot', 'png_file': 'dt.png', 'dot cmd rc': 0}

![dt.png](dt.png)

In [75]:
(X_train_dt, y_train_dt) = (th_dt.X_train, th_dt.y_train)

y_train_dt.value_counts()
y_train_dt[ X_train_dt[ "Sex" ] < 0.5 ].value_counts()
y_train_dt[ (X_train_dt[ "Sex"] < 0.5) & ( X_train_dt[ "Age" ] <= 6.5)    ].value_counts()

0    549
1    342
Name: Survived, dtype: int64

0    468
1    109
Name: Survived, dtype: int64

1    16
0     8
Name: Survived, dtype: int64

In [92]:
X_f, y_f, X_fn, y_fn   = th_dt.partition(X_train_dt, y_train_dt, [ ("Sex", 0.5 ) ] )
y_f.value_counts()

X_f2, y_f2, X_f2n, y_f2n = th_dt.partition(X_train_dt, y_train_dt, [ ("Sex", 0.5 ), ("Age", 6.5) ] )
y_f2.value_counts()

# Run the 2 level i 1 step increments
X_f21, y_f21, X_f21n, y_f21n = th_dt.partition( X_f, y_f, [ ("Age", 6.5)])
y_f21.value_counts()
y_f21n.value_counts()

Filtering column Sex on 0.5


0    468
1    109
Name: Survived, dtype: int64

Filtering column Sex on 0.5
Filtering column Age on 6.5


1    16
0     8
Name: Survived, dtype: int64

Filtering column Age on 6.5


1    16
0     8
Name: Survived, dtype: int64

0    460
1     93
Name: Survived, dtype: int64

In [88]:
y_f2.value_counts()

1    16
0     8
Name: Survived, dtype: int64